### Nodes for README:
- Treatment of PAR_Y regions in short id
- fields in index, ensembl equivalence
- default paths if not provided
- default assembly (GRCh19 < Gencode 20, GRCh38 >= 20)

### TODO:
- Implement approximate search by name

In [6]:
import sys
import os
from os import path,listdir
import subprocess
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

import re 
import gzip
import networkx as nx
from networkx.algorithms.traversal.depth_first_search import dfs_tree
from networkx.drawing.nx_agraph import graphviz_layout
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

In [12]:
##################################
### General methods
##################################

def Call(s):
    subprocess.call(s, shell=True, executable='/bin/bash')
    
    
def bed_sort(input_file):
    call = "sort -k1,1 -k2,2n -o '%s' '%s'"%(input_file,input_file)    
    Call(call)
    
    
def gtf_sort(input_file):
    call = "sort -k1,1 -k4,4n -o '%s' '%s'"%(input_file,input_file)    
    Call(call)
        
        
def microparser(line):
    if len(line.strip())==0:
        return None
    X = line.strip().split('\t')        
    r = {a.split(' ')[0]:a.split(' ')[1].replace('"','') for a in [x.strip() for x in X[8].split(';') if len(x.strip())>0]}
    r.update({'chr':X[0],'start':X[3],'end':X[4],'strand':X[6],'feature':X[2]})
    return r


def parse_meta(text):
    assembly = re.findall('human genome \((.*)\),',text)[0].strip()
    version = re.findall('version (.*) \(',text)[0].strip()
    ensembl = re.findall('Ensembl (.*)\)',text)[0].strip()
    _date = re.findall('date: (.*)',text)[0].strip()
    return {'gencode':version,'assembly':assembly,'ensembl':ensembl,'date':_date}


##################################
### Main class definition
##################################

class GHTracker:
    def __init__(self,download_path=None,index_path=None,LATEST_VERSION=37):
        self.download_path = download_path
        self.index_path = index_path
        self.LATEST_VERSION = LATEST_VERSION
    
    ##################################
    ### Class properties
    ##################################
    @property
    def download_path(self):        
        return self._download_path
    
    @download_path.setter
    def download_path(self,download_path):
        if download_path is None:
            d = path.abspath(os.getcwd())            
        else:
            d = path.abspath(download_path)
        self._download_path = d
        self.raw_path = path.join(self.download_path,'raw')
        self.gene_path = path.join(self.download_path,'gene')
        self.meta_path = path.join(self.download_path,'meta')
#         for folder in [self.download_path, self.raw_path, self.gene_path, self.meta_path]:
#             if not path.exists(folder):
#                 os.makedirs()        
        
    @property
    def index_path(self):        
        return self._index_path
    
    @index_path.setter
    def index_path(self,index_path):
        if index_path is None:
            d = path.abspath(os.getcwd())
        else:
            d = path.abspath(index_path)
        #Only reload data structures if new path is different
        if (hasattr(self, '_index_path')) and (self._index_path == d):
            return
        self._index_path = d        
        self.ensembl_eq = path.join(self.index_path,'ensembl_equivalence.csv')
        self.gene_index = path.join(self.index_path,'gene_index.csv.gz')
        self.gene_graph = path.join(self.index_path,'gene_network.gml')
        self.df_index = None
        self.df_index_last = None
        self.df_ensembl = None
        self.G = None
        self.GR = None
        
    def load_index(self,index_path=None):
        if index_path is not None:
            self.index_path = index_path
        #TODO: CHECK IF INDEX FILES EXIST
        # Load if is not loaded before or index_path changed
        if self.df_index is None:
            df = pd.read_csv(self.gene_index)
            df = df[['gene_shortid','gene_id','gene_name','hgnc_id','gene_type','coord','gencode']]
            df2 = pd.read_csv(self.ensembl_eq)
            df = pd.merge(df,df2,left_on='gencode',right_on='gencode')
            dfx = df.sort_values(by=['gene_shortid','gencode'],ascending=[True,False]).drop_duplicates(subset=['gene_shortid'],keep='first')
            self.df_index = df
            self.df_index_last = dfx
            self.df_ensembl = df2
        print('Entries:',len(self.df_index))
        
    def load_graph(self,index_path=None):
        if index_path is not None:
            self.index_path = index_path
        #TODO: CHECK IF INDEX FILES EXIST
        # Load if is not loaded before or index_path changed
        if self.G is None:
            G = nx.read_gml(self.gene_graph)
            GR = G.reverse()
            self.G = G
            self.GR = GR
        print('Nodes:',len(self.G.nodes()),'Edges:',len(self.G.edges()))
        
    ##################################
    ### Query methods
    ##################################
    def query_by_geneid_list(self,gene_list,output_tsv=None):
        DF = self.df_index_last
        LATEST_VERSION = self.LATEST_VERSION
        gene_set = set(gene_list)
        dfx = DF[(DF['gene_id'].isin(gene_set))|(DF['gene_shortid'].isin(gene_set))].copy()    
        dfx['deprecated'] = dfx['gencode'].apply(lambda x:int(x!=LATEST_VERSION))
        dfx = dfx.reset_index(drop=True)
        if output_tsv is not None:
            dfx.to_csv(output_tsv,sep='\t',index=None)
        return dfx

    def query_geneid_history(self,gene_id,output_tsv=None):
        DF = self.df_index
        dfx = DF[(DF['gene_id']==gene_id)|(DF['gene_shortid']==gene_id)].copy()    
        dfx = dfx.sort_values(by=['gene_shortid','gencode'],ascending=[True,False])    
        dfx = dfx.reset_index(drop=True)
        if output_tsv is not None:
            dfx.to_csv(output_tsv,sep='\t',index=None)
        return dfx

    def query_genename_history(self,gene_name,output_tsv=None):
        DF = self.df_index
        dfx = DF[(DF['gene_name']==gene_name)].copy()
        short_ids = set(dfx['gene_shortid'].unique())
        dfx = DF[DF['gene_shortid'].isin(short_ids)]
        dfx = dfx.sort_values(by=['gene_shortid','gencode'],ascending=[True,False])
        dfx = dfx.reset_index(drop=True)
        if output_tsv is not None:
            dfx.to_csv(output_tsv,sep='\t',index=None)
        return dfx

    def putative_mappings(self,gene_list):
        G = self.G
        R = {}
        for gene in gene_list:
            g1 = dfs_tree(G,gene)        
            latest = None
            for x in g1.nodes():
                a = G.nodes()[x]
                #print(x,a)
                if a['status']=='latest':        
                    if latest is None:
                        latest=[x]
                    else:
                        latest.append(x)        
            #R[gene] = '' if latest is None else ','.join(latest)
            R[gene] = set() if latest is None else set(latest)
        #dfx = pd.DataFrame.from_dict(R,orient='index').reset_index()
        #dfx.columns = ['query','mappings']
        #return dfx
        return R
    
    ##################################
    ### Download methods
    ##################################
    
    
    ##################################
    ### Building methods
    ##################################

In [13]:
GH = GHTracker(index_path='index')

In [14]:
GH.load_index()
GH.load_graph()

Entries: 1888250
Nodes: 74720 Edges: 3156


---

In [16]:
gene_list = [
    'ENSG00000230544',
    'ENSG00000240453',
    'ENSG00000254100',
    'ENSG00000259484',
    'ENSG00000259758',
    'ENSG00000260940',
    'ENSG00000261496',
    'ENSG00000261685',
    'ENSG00000266584',
    'ENSG00000122952'
]

In [17]:
GH.query_by_geneid_list(gene_list)

,gene_shortid,gene_id,gene_name,hgnc_id,gene_type,coord,gencode,ensembl,assembly,date,deprecated
0,ENSG00000122952,ENSG00000122952.17,ZWINT,HGNC:13195,protein_coding,chr10:56357227-56361273(-),37,103,GRCh38,2020-12-07,0
1,ENSG00000230544,ENSG00000230544.1,LINC00453,NaN,lincRNA,chr13:114586640-114588308(+),18,73,GRCh37,2013-09-02,1
2,ENSG00000240453,ENSG00000240453.1,RP11-206L10.10,NaN,processed_transcript,chr1:810109-817712(-),20,76,GRCh38,2014-08-26,1
3,ENSG00000254100,ENSG00000254100.1,AC069120.3,NaN,lincRNA,chr8:38552248-38559020(-),27,90,GRCh38,2017-08-01,1
4,ENSG00000259484,ENSG00000259484.1,RP11-323F24.1,NaN,processed_transcript,chr15:57151866-57210697(-),17,72,GRCh37,2013-06-17,1
5,ENSG00000259758,ENSG00000259758.1,CASC7,NaN,lincRNA,chr8:140520156-140529501(-),21,77,GRCh38,2014-09-29,1
6,ENSG00000260940,ENSG00000260940.1,RP4-575N6.5,NaN,sense_overlapping,chr1:101243158-101243749(+),28,92,GRCh38,2018-03-23,1
7,ENSG00000261496,ENSG00000261496.1,RP13-514E23.1,NaN,sense_overlapping,chr4:86012296-86013874(-),25,85,GRCh38,2016-07-15,1
8,ENSG00000261685,ENSG00000261685.2,RP11-401P9.4,NaN,lincRNA,chr16:50645809-50649249(+),24,83,GRCh38,2015-12-03,1
9,ENSG00000266584,ENSG00000266584.1,AL355149.1,NaN,miRNA,chr1:16548914-16548987(+),24,83,GRCh38,2015-12-03,1


---
---

In [140]:
def query_by_geneid_list(DF,LATEST_VERSION,gene_list,output_tsv=None):
    gene_set = set(gene_list)
    dfx = DF[(DF['gene_id'].isin(gene_set))|(DF['gene_shortid'].isin(gene_set))].copy()    
    dfx['deprecated'] = dfx['gencode'].apply(lambda x:int(x!=LATEST_VERSION))
    dfx = dfx.reset_index(drop=True)
    if output_tsv is not None:
        dfx.to_csv(output_tsv,sep='\t',index=None)
    return dfx

def query_geneid_history(DF,gene_id,output_tsv=None):
    dfx = DF[(DF['gene_id']==gene_id)|(DF['gene_shortid']==gene_id)].copy()    
    dfx = dfx.sort_values(by=['gene_shortid','gencode'],ascending=[True,False])    
    dfx = dfx.reset_index(drop=True)
    if output_tsv is not None:
        dfx.to_csv(output_tsv,sep='\t',index=None)
    return dfx

def query_genename_history(DF,gene_name,output_tsv=None):
    dfx = DF[(DF['gene_name']==gene_name)].copy()
    short_ids = set(dfx['gene_shortid'].unique())
    dfx = DF[DF['gene_shortid'].isin(short_ids)]
    dfx = dfx.sort_values(by=['gene_shortid','gencode'],ascending=[True,False])
    dfx = dfx.reset_index(drop=True)
    if output_tsv is not None:
        dfx.to_csv(output_tsv,sep='\t',index=None)
    return dfx

def putative_mappings(G,gene_list):
    R = {}
    for gene in gene_list:
        g1 = dfs_tree(G,gene)        
        latest = None
        for x in g1.nodes():
            a = G.nodes()[x]
            #print(x,a)
            if a['status']=='latest':        
                if latest is None:
                    latest=[x]
                else:
                    latest.append(x)        
        #R[gene] = '' if latest is None else ','.join(latest)
        R[gene] = set() if latest is None else set(latest)
    #dfx = pd.DataFrame.from_dict(R,orient='index').reset_index()
    #dfx.columns = ['query','mappings']
    #return dfx
    return R

In [142]:
#query_by_geneid_list(GH.df_index_last,GH.LATEST_VERSION,gene_list)
#query_geneid_history(GH.df_index,'ENSG00000251562')
#query_genename_history(GH.df_index,'MALAT1')
R = putative_mappings(GH.G,gene_list)

In [144]:
for id_query,v in R.items():
    for id_mapping in v:
        print(id_query,id_mapping)

ENSG00000240453 ENSG00000230092
ENSG00000254100 ENSG00000196166
ENSG00000254100 ENSG00000253361
ENSG00000259484 ENSG00000137871
ENSG00000122952 ENSG00000122952
